Here we combine all the cleaned data to create a master table, on which we train the data for predictive modelling.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = 'EPIC_EMR_cleaned/'
information = pd.read_csv(os.path.join(path,'patient_information_cleaned.csv'))
information = information.drop_duplicates()

In [3]:
information.head()

,LOG_ID,MRN,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,LOS,SURGERY_DATE,BIRTH_DATE,WEIGHT,IN_OR_DTTM,OUT_OR_DTTM,AN_START_DATETIME,AN_STOP_DATETIME,ICU_ADMIN_FLAG_encoded,SEX_encoded,PRIMARY_ANES_TYPE_NM_encoded,PATIENT_CLASS_GROUP_encoded,DISCH_DISP_encoded,ASA_RATING_encoded,HEIGHT_IN_CM
0,6af225abbb809cec,7c32ed97b90276e9,10/28/20 9:28,11/5/20 18:52,8.0,10/28/20 0:00,55,2620.83,10/28/20 12:38,10/28/20 16:02,10/28/20 12:38,10/28/20 16:08,0,2,10,0,20.0,3.0,152.40
1,3119d72120dacb7c,cdc958fff5570ab0,4/7/19 5:18,4/8/19 9:50,1.0,4/7/19 0:00,54,2125.23,4/7/19 7:12,4/7/19 13:38,4/7/19 7:12,4/7/19 13:43,0,2,1,1,15.0,2.0,157.48
2,f0d16bb8e593218e,95d54ee9f5abfab3,12/14/18 7:16,12/16/18 16:05,2.0,12/14/18 0:00,43,3710.78,12/14/18 8:44,12/14/18 12:14,12/14/18 8:46,12/14/18 12:30,1,1,1,1,15.0,2.0,180.34
3,32f1980e3944170e,0dcace1cd2a4d89d,6/22/19 8:10,6/23/19 11:15,1.0,6/22/19 0:00,79,1968.27,6/22/19 10:36,6/22/19 13:05,6/22/19 10:36,6/22/19 13:12,1,1,1,1,15.0,3.0,NaN
4,db83846371c937a9,e3025ead155cb9bb,10/12/19 8:43,10/13/19 18:02,1.0,10/12/19 0:00,67,2567.92,10/12/19 10:38,10/12/19 14:31,10/12/19 10:38,10/12/19 14:37,1,2,1,1,15.0,2.0,163.83


In [4]:
#Convert all date columns stored as strings into datetime format
#Create a new column for anesthesia duration from AN-Start and AN-Stop
information['AN_START_DATETIME'] = pd.to_datetime(information['AN_START_DATETIME'],format='%m/%d/%y %H:%M')
information['AN_STOP_DATETIME'] = pd.to_datetime(information['AN_STOP_DATETIME'],format='%m/%d/%y %H:%M')
information['IN_OR_DTTM'] = pd.to_datetime(information['IN_OR_DTTM'],format='%m/%d/%y %H:%M')
information['AN_DURATION'] = (information['AN_STOP_DATETIME'] - information['AN_START_DATETIME']).dt.seconds / 3600
information.head()

,LOG_ID,MRN,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,LOS,SURGERY_DATE,BIRTH_DATE,WEIGHT,IN_OR_DTTM,OUT_OR_DTTM,AN_START_DATETIME,AN_STOP_DATETIME,ICU_ADMIN_FLAG_encoded,SEX_encoded,PRIMARY_ANES_TYPE_NM_encoded,PATIENT_CLASS_GROUP_encoded,DISCH_DISP_encoded,ASA_RATING_encoded,HEIGHT_IN_CM,AN_DURATION
0,6af225abbb809cec,7c32ed97b90276e9,10/28/20 9:28,11/5/20 18:52,8.0,10/28/20 0:00,55,2620.83,2020-10-28 12:38:00,10/28/20 16:02,2020-10-28 12:38:00,2020-10-28 16:08:00,0,2,10,0,20.0,3.0,152.40,3.500000
1,3119d72120dacb7c,cdc958fff5570ab0,4/7/19 5:18,4/8/19 9:50,1.0,4/7/19 0:00,54,2125.23,2019-04-07 07:12:00,4/7/19 13:38,2019-04-07 07:12:00,2019-04-07 13:43:00,0,2,1,1,15.0,2.0,157.48,6.516667
2,f0d16bb8e593218e,95d54ee9f5abfab3,12/14/18 7:16,12/16/18 16:05,2.0,12/14/18 0:00,43,3710.78,2018-12-14 08:44:00,12/14/18 12:14,2018-12-14 08:46:00,2018-12-14 12:30:00,1,1,1,1,15.0,2.0,180.34,3.733333
3,32f1980e3944170e,0dcace1cd2a4d89d,6/22/19 8:10,6/23/19 11:15,1.0,6/22/19 0:00,79,1968.27,2019-06-22 10:36:00,6/22/19 13:05,2019-06-22 10:36:00,2019-06-22 13:12:00,1,1,1,1,15.0,3.0,NaN,2.600000
4,db83846371c937a9,e3025ead155cb9bb,10/12/19 8:43,10/13/19 18:02,1.0,10/12/19 0:00,67,2567.92,2019-10-12 10:38:00,10/12/19 14:31,2019-10-12 10:38:00,2019-10-12 14:37:00,1,2,1,1,15.0,2.0,163.83,3.983333


In [5]:
#Remove all null values in IN_OR_DTTM column
information = information[information['IN_OR_DTTM'].notna()]
log_id_cutoff = information[['LOG_ID','MRN','IN_OR_DTTM']].copy(deep=True)
print(log_id_cutoff.shape)

(57959, 3)


In [6]:
log_id_cutoff.head()

,LOG_ID,MRN,IN_OR_DTTM
0,6af225abbb809cec,7c32ed97b90276e9,2020-10-28 12:38:00
1,3119d72120dacb7c,cdc958fff5570ab0,2019-04-07 07:12:00
2,f0d16bb8e593218e,95d54ee9f5abfab3,2018-12-14 08:44:00
3,32f1980e3944170e,0dcace1cd2a4d89d,2019-06-22 10:36:00
4,db83846371c937a9,e3025ead155cb9bb,2019-10-12 10:38:00


In [7]:
path = 'EPIC_EMR_cleaned/'
labs = pd.read_csv(os.path.join(path,'patient_labs_cleaned.csv'))
labs = labs.drop_duplicates()
labs.isna().sum()

Abnormal Flag_encoded    0
Collection Datetime      0
LOG_ID                   0
MRN                      0
dtype: int64

In [8]:
labs.head()

,Abnormal Flag_encoded,Collection Datetime,LOG_ID,MRN
0,1,2020-03-10 23:36:00,3ab8e6344cc9f954,559c869f9d7db8ee
1,1,2020-03-08 04:09:00,3ab8e6344cc9f954,559c869f9d7db8ee
2,1,2020-03-09 05:33:00,3ab8e6344cc9f954,559c869f9d7db8ee
3,1,2020-03-12 21:36:00,3ab8e6344cc9f954,559c869f9d7db8ee
5,4,2020-03-08 02:36:00,3ab8e6344cc9f954,559c869f9d7db8ee


In [9]:
labs = labs.merge(log_id_cutoff[['LOG_ID','IN_OR_DTTM']],how='inner',on='LOG_ID')#Join Labs table and LOGS ID table on Log_ID
labs['Collection Datetime'] = pd.to_datetime(labs['Collection Datetime'])#Convert Collection Datetime from string to datetime
labs = labs[labs['Collection Datetime'] <= labs['IN_OR_DTTM']]
labs.head()

,Abnormal Flag_encoded,Collection Datetime,LOG_ID,MRN,IN_OR_DTTM
0,1,2020-03-10 23:36:00,3ab8e6344cc9f954,559c869f9d7db8ee,2020-03-12 08:04:00
1,1,2020-03-08 04:09:00,3ab8e6344cc9f954,559c869f9d7db8ee,2020-03-12 08:04:00
2,1,2020-03-09 05:33:00,3ab8e6344cc9f954,559c869f9d7db8ee,2020-03-12 08:04:00
4,4,2020-03-08 02:36:00,3ab8e6344cc9f954,559c869f9d7db8ee,2020-03-12 08:04:00
5,5,2020-03-07 23:33:00,3ab8e6344cc9f954,559c869f9d7db8ee,2020-03-12 08:04:00


Store the number of Abnormal flags recorded for each Patient Logs

In [10]:
list(labs['Abnormal Flag_encoded'].unique())

[1, 4, 5, 2, 0, 3]

In [11]:
lab_report_df = pd.DataFrame(index=labs['LOG_ID'].unique())
for encoding in list(labs['Abnormal Flag_encoded'].unique()):
    labs_temp = labs[labs['Abnormal Flag_encoded'] == encoding]
    lab_report_df['Num_labs_'+str(encoding)+'_count'] = labs_temp.groupby(by='LOG_ID')['Abnormal Flag_encoded'].count()
lab_report_df.head(10)


,Num_labs_1_count,Num_labs_4_count,Num_labs_5_count,Num_labs_2_count,Num_labs_0_count,Num_labs_3_count
3ab8e6344cc9f954,41.0,29.0,30.0,6.0,NaN,NaN
f07e166781abd026,60.0,50.0,46.0,1.0,12.0,2.0
dbb42e6801cf1e62,77.0,57.0,83.0,5.0,16.0,4.0
3a19916d5a0dc3ec,25.0,18.0,14.0,1.0,NaN,NaN
6f0aadfadfce3e20,17.0,9.0,11.0,1.0,NaN,NaN
0b31b1841a16cfc5,303.0,242.0,258.0,8.0,8.0,6.0
b40a80aaf894a847,17.0,5.0,7.0,3.0,NaN,NaN
34031ce6a0d152a7,6.0,2.0,3.0,1.0,NaN,1.0
fedac74965a05b3f,9.0,3.0,4.0,2.0,NaN,NaN
ce1f1dd5a63a8f30,15.0,9.0,8.0,NaN,1.0,NaN


In [12]:
path = 'EPIC_EMR_cleaned/'
history = pd.read_csv(os.path.join(path,'patient_history_cleaned.csv'))
history = history.drop_duplicates()

In [13]:
history.head()

,mrn,dx_name_encoded
0,cdce1f368362568f,32990
1,309411c5b6b9658e,45319
2,13e76a6d564f8049,37245
3,f42b6b0d38e95a7d,37245
4,ef7b3e727d8cc736,37245


In [14]:
#Store the number of visits of each patients
num_of_past_visits = history.groupby('mrn')['dx_name_encoded'].count().sort_values(ascending=False).reset_index()
num_of_past_visits = num_of_past_visits.rename(columns={"mrn":'MRN','dx_name_encoded':'Num_Past_Visits'})
num_of_past_visits.head()

,MRN,Num_Past_Visits
0,121590d818fc810e,175
1,477e792e1ae54cf2,169
2,e63a4d4458fca74a,163
3,cb0719ad7fd07fcf,155
4,5a5b4f6c2312d4fa,142


In [15]:
path = 'EPIC_EMR_cleaned/'
procedure_event = pd.read_csv(os.path.join(path,'patient_procedure_events_cleaned.csv'))
procedure_event = procedure_event.drop_duplicates()

In [16]:
procedure_event.head()

,LOG_ID,MRN,EVENT_TIME,EVENT_DISPLAY_NAME_encoded
0,e1a066d449ed18b0,499386c1a8693fd0,4/27/19 3:04,72
1,068b548cf0d91976,68d58e4c2dcd06f6,5/6/19 11:46,72
2,05195f764521086f,85a062b39530bf42,6/12/19 20:59,27
3,33a907e9affc529f,f7bf5c74f687a530,9/23/19 7:57,55
4,59dc47ed0b52c5d8,424d7ceb87a7defd,9/3/19 7:24,29


In [17]:
procedure_event = procedure_event.merge(log_id_cutoff[['LOG_ID','IN_OR_DTTM']],how='inner',on='LOG_ID')
procedure_event['EVENT_TIME'] = pd.to_datetime(procedure_event['EVENT_TIME'],format='%m/%d/%y %H:%M')
procedure_event = procedure_event[procedure_event['EVENT_TIME'] <= procedure_event['IN_OR_DTTM']]

Store the number of times each procedure is performed for each Patient Logs

In [18]:
procedure_features = pd.DataFrame(index=procedure_event['LOG_ID'].unique())
for encoding in list(procedure_event['EVENT_DISPLAY_NAME_encoded'].unique()):
    procedure_temp = procedure_event[procedure_event['EVENT_DISPLAY_NAME_encoded'] == encoding]
    procedure_features['Num_procedure_'+str(encoding)+'_count'] = procedure_temp.groupby(by='LOG_ID')['EVENT_DISPLAY_NAME_encoded'].count()

In [19]:
procedure_features.head()

,Num_procedure_85_count,Num_procedure_49_count,Num_procedure_89_count,Num_procedure_55_count,Num_procedure_72_count,Num_procedure_79_count,Num_procedure_50_count,Num_procedure_21_count,Num_procedure_88_count,Num_procedure_71_count,...,Num_procedure_44_count,Num_procedure_84_count,Num_procedure_30_count,Num_procedure_63_count,Num_procedure_32_count,Num_procedure_9_count,Num_procedure_10_count,Num_procedure_80_count,Num_procedure_8_count,Num_procedure_53_count
e1a066d449ed18b0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
068b548cf0d91976,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05195f764521086f,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33a907e9affc529f,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59dc47ed0b52c5d8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
path = 'EPIC_EMR_cleaned/'
post_op = pd.read_csv(os.path.join(path,'patient_post_op_complication_cleaned.csv'))
post_op = post_op.drop(columns=['SMRTDTA_ELEM_VALUE_encoded','CONTEXT_NAME_encoded'])
post_op = post_op.drop_duplicates()

In [21]:
post_op.head()

,Element_abbr_encoded,LOG_ID,MRN
0,2,a8b3f9afa63c96d1,aaebaad9587efc99
1,2,5dc0d50fa1d11ea2,dc6e29201905577b
2,2,c66c0c773c2280d8,66d9279ff5f4b927
3,2,bec65b5c7ae241b8,c80dbde76c38ecad
4,2,104d64f326946e24,bb42a6512d570bbc


Store the number of times each post operative complication recorded for each Patient Logs

In [22]:
post_op_features = pd.DataFrame(index=post_op['LOG_ID'].unique())
for encoding in list(post_op['Element_abbr_encoded'].unique()):
    post_op_temp = post_op[post_op['Element_abbr_encoded'] == encoding]
    post_op_features['Num_Post_OP_'+str(encoding)+'_count'] = post_op_temp.groupby(by='LOG_ID')['Element_abbr_encoded'].count()

In [23]:
post_op_features.head()

,Num_Post_OP_2_count,Num_Post_OP_11_count,Num_Post_OP_9_count,Num_Post_OP_7_count,Num_Post_OP_6_count,Num_Post_OP_3_count,Num_Post_OP_0_count,Num_Post_OP_4_count,Num_Post_OP_1_count,Num_Post_OP_5_count,Num_Post_OP_8_count,Num_Post_OP_10_count
a8b3f9afa63c96d1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5dc0d50fa1d11ea2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c66c0c773c2280d8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bec65b5c7ae241b8,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
104d64f326946e24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
final_dataset = information[['LOG_ID',"MRN",'LOS','BIRTH_DATE','WEIGHT','ICU_ADMIN_FLAG_encoded','SEX_encoded',
'PRIMARY_ANES_TYPE_NM_encoded','PATIENT_CLASS_GROUP_encoded','ASA_RATING_encoded','HEIGHT_IN_CM',]].copy(deep=True)# remove an duration to prevent data leakage
final_dataset = final_dataset.drop_duplicates(subset=['LOG_ID'],keep='first')
print(final_dataset.shape)

(57952, 11)


In [25]:
final_dataset.head(10)

,LOG_ID,MRN,LOS,BIRTH_DATE,WEIGHT,ICU_ADMIN_FLAG_encoded,SEX_encoded,PRIMARY_ANES_TYPE_NM_encoded,PATIENT_CLASS_GROUP_encoded,ASA_RATING_encoded,HEIGHT_IN_CM
0,6af225abbb809cec,7c32ed97b90276e9,8.0,55,2620.83,0,2,10,0,3.0,152.40
1,3119d72120dacb7c,cdc958fff5570ab0,1.0,54,2125.23,0,2,1,1,2.0,157.48
2,f0d16bb8e593218e,95d54ee9f5abfab3,2.0,43,3710.78,1,1,1,1,2.0,180.34
3,32f1980e3944170e,0dcace1cd2a4d89d,1.0,79,1968.27,1,1,1,1,3.0,NaN
4,db83846371c937a9,e3025ead155cb9bb,1.0,67,2567.92,1,2,1,1,2.0,163.83
5,f0aafb8dcb6d1949,e177fdf1ed283ffe,0.0,51,1922.41,1,2,1,1,2.0,NaN
6,aea32165b53d1c29,1f9ced1dbc1f9ed4,0.0,47,5114.67,1,2,3,1,3.0,172.72
7,e8aba3cde5cddc4b,50baf65ac33dac67,0.0,78,2528.00,1,1,6,1,NaN,180.34
8,b52faa639b25b425,b3796981839ce195,1.0,68,3232.83,1,1,1,1,3.0,167.64
9,74fd8b1a2ee73ae3,85bb7b2dfac56ca7,6.0,57,3116.42,0,2,1,0,3.0,NaN


In [26]:
final_dataset = final_dataset.merge(lab_report_df,right_index=True,left_on='LOG_ID',how='left')
final_dataset = final_dataset.merge(num_of_past_visits,on='MRN',how='left')
final_dataset = final_dataset.merge(procedure_features,right_index=True,left_on='LOG_ID',how='left')
#final_dataset = final_dataset.merge(post_op_features,right_index=True,left_on='LOG_ID',how='left')
final_dataset.shape #Avoiding post opeartive features because it is an event after the surgery.

(57952, 59)

In [27]:
final_dataset.head(10)

,LOG_ID,MRN,LOS,BIRTH_DATE,WEIGHT,ICU_ADMIN_FLAG_encoded,SEX_encoded,PRIMARY_ANES_TYPE_NM_encoded,PATIENT_CLASS_GROUP_encoded,ASA_RATING_encoded,...,Num_procedure_44_count,Num_procedure_84_count,Num_procedure_30_count,Num_procedure_63_count,Num_procedure_32_count,Num_procedure_9_count,Num_procedure_10_count,Num_procedure_80_count,Num_procedure_8_count,Num_procedure_53_count
0,6af225abbb809cec,7c32ed97b90276e9,8.0,55,2620.83,0,2,10,0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3119d72120dacb7c,cdc958fff5570ab0,1.0,54,2125.23,0,2,1,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f0d16bb8e593218e,95d54ee9f5abfab3,2.0,43,3710.78,1,1,1,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32f1980e3944170e,0dcace1cd2a4d89d,1.0,79,1968.27,1,1,1,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,db83846371c937a9,e3025ead155cb9bb,1.0,67,2567.92,1,2,1,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,f0aafb8dcb6d1949,e177fdf1ed283ffe,0.0,51,1922.41,1,2,1,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,aea32165b53d1c29,1f9ced1dbc1f9ed4,0.0,47,5114.67,1,2,3,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,e8aba3cde5cddc4b,50baf65ac33dac67,0.0,78,2528.00,1,1,6,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,b52faa639b25b425,b3796981839ce195,1.0,68,3232.83,1,1,1,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,74fd8b1a2ee73ae3,85bb7b2dfac56ca7,6.0,57,3116.42,0,2,1,0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
final_dataset.to_feather('final_dataset.feather')